## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-12-12-59-51 +0000,nyt,Freed South Korean Workers Return From the U.S...,https://www.nytimes.com/2025/09/12/world/asia/...
1,2025-09-12-12-57-00 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2025/09/12/us/cha...
2,2025-09-12-12-55-09 +0000,bbc,A palm print and a figure in black - what we k...,https://www.bbc.com/news/articles/cy04p4x21e5o...
3,2025-09-12-12-51-53 +0000,wapo,Live updates: Trump says Charlie Kirk shooting...,https://www.washingtonpost.com/nation/2025/09/...
4,2025-09-12-12-50-11 +0000,nypost,New Ernst bill would deactivate gov’t-issued c...,https://nypost.com/2025/09/12/us-news/new-joni...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2312/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
19,kirk,62
18,charlie,53
17,trump,42
20,shooting,16
23,new,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
88,2025-09-12-09-30-00 +0000,wsj,"Charlie Kirk and JD Vance spoke regularly, tal...",https://www.wsj.com/politics/elections/jd-vanc...,212
199,2025-09-11-23-04-00 +0000,wsj,President Trump’s team began putting in place ...,https://www.wsj.com/politics/elections/angry-a...,206
30,2025-09-12-12-07-11 +0000,nypost,"Tyler Robinson, 22-year-old from Utah, ID’d as...",https://nypost.com/2025/09/12/us-news/we-have-...,200
284,2025-09-11-18-02-59 +0000,nyt,Trump Says He Will Award Charlie Kirk the Pres...,https://www.nytimes.com/2025/09/11/us/charlie-...,188
10,2025-09-12-12-40-01 +0000,startribune,Trump says 'with a high degree of certainty' t...,https://www.startribune.com/officials-plead-fo...,188


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
88,212,2025-09-12-09-30-00 +0000,wsj,"Charlie Kirk and JD Vance spoke regularly, tal...",https://www.wsj.com/politics/elections/jd-vanc...
199,91,2025-09-11-23-04-00 +0000,wsj,President Trump’s team began putting in place ...,https://www.wsj.com/politics/elections/angry-a...
247,53,2025-09-11-20-18-21 +0000,bbc,Bolsonaro's coup trial gripped Brazil - and hi...,https://www.bbc.com/news/articles/cj3yekj2xygo...
104,53,2025-09-12-08-38-23 +0000,nypost,Prince Harry makes surprise visit to Kyiv as h...,https://nypost.com/2025/09/12/world-news/princ...
126,53,2025-09-12-06-29-26 +0000,wapo,Officials release new video of person fleeing ...,https://www.washingtonpost.com/nation/2025/09/...
167,43,2025-09-12-01-10-28 +0000,bbc,Man accused of Trump assassination attempt cut...,https://www.bbc.com/news/articles/c8d793z4mzgo...
249,43,2025-09-11-20-10-31 +0000,nypost,Russian drone incursion over Poland was ‘delib...,https://nypost.com/2025/09/11/world-news/russi...
155,39,2025-09-12-02-58-50 +0000,nyt,South Korean Leader Warns of U.S. Investments ...,https://www.nytimes.com/2025/09/11/world/asia/...
107,38,2025-09-12-08-21-06 +0000,nypost,"Josh Kraft, son of billionaire Patriots owner,...",https://nypost.com/2025/09/12/us-news/robert-k...
63,36,2025-09-12-10-18-00 +0000,wsj,Paramount Skydance Prepares Ellison-Backed Bid...,https://www.wsj.com/business/media/paramount-s...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
